In [ ]:
!python -m spacy download en
!pip install transformers
!pip install torch
!pip install tensorflow
!python -m spacy download en_core_web_lg

In [2]:
import numpy as np
import networkx as nx
import spacy
from summa import keywords
from summa.summarizer import summarize

# Laden des Spacy-Modells
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline
import torch

In [18]:
df_train=pd.read_csv('data/data_train_with_features.csv')
df_train = df_train.drop('Unnamed: 0', axis=1)
df_train.head()

,classification,text,sentences,num_sentences,words,num_words,lemmas,stops,num_stops
0,Scientific,the interest in anchoring phenomena and phenom...,['the interest in anchoring phenomena and phen...,222,"['interest', 'anchoring', 'phenomena', 'phenom...",3533,"['interest', 'anchor', 'phenomenon', 'phenomen...","[the, in, and, in, has, been, by, their, in, a...",2426
1,Scientific,"galaxy clusters , as the largest peaks in the ...","['galaxy clusters , as the largest peaks in th...",372,"['galaxy', 'clusters', 'largest', 'peaks', 'co...",8982,"['galaxy', 'cluster', 'large', 'peak', 'cosmic...","[as, the, in, the, an, in, and, in, for, and, ...",5768
2,Scientific,quantum correlations between components of a s...,['quantum correlations between components of a...,138,"['quantum', 'correlations', 'components', 'sys...",2451,"['quantum', 'correlation', 'component', 'syste...","[between, of, a, or, are, on, the, of, whole, ...",1780
3,Scientific,methanol masers are often found in star - form...,['methanol masers are often found in star - fo...,141,"['methanol', 'masers', 'found', 'star', 'formi...",3232,"['methanol', 'maser', 'find', 'star', 'form', ...","[are, often, in, are, two, of, to, i, most, th...",1940
4,Scientific,interdisciplinary research has recently gained...,['interdisciplinary research has recently gain...,88,"['interdisciplinary', 'research', 'recently', ...",2695,"['interdisciplinary', 'research', 'recently', ...","[has, a, in, the, of, to, the, in, was, to, tw...",1997


In [9]:





def textrank_extractive(text, compression_rate=0.5):
    # Tokenisierung
    nlp = spacy.load("en_core_web_lg")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    # Speichern der Spacy-Dokumente der Sätze für spätere Verwendung
    sentence_docs = [nlp(sentence) for sentence in sentences]

    # Extrahiere Schlüsselsätze mit TextRank
    num_sentences = max(1, int(len(sentences) * compression_rate))
    extracted_sentences = summarize(text, words=num_sentences, split=True)

    # Erzeuge eine Matrix mit den Ähnlichkeiten zwischen den Sätzen
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i, doc_i in enumerate(sentence_docs):
        for j, doc_j in enumerate(sentence_docs):
            similarity = similarity_function(doc_i, doc_j)
            similarity_matrix[i, j] = similarity

    # Konstruiere einen Graphen mit den Sätzen als Knoten und den Ähnlichkeiten als Kanten
    graph = nx.from_numpy_array(similarity_matrix)

    # Berechne den TextRank-Score für jeden Satz
    scores = nx.pagerank(graph)

    # Wähle die besten Sätze basierend auf ihren TextRank-Scores aus
    top_sentences = sorted(scores, key=scores.get, reverse=True)[:num_sentences]

    # Sortiere die ausgewählten Sätze nach ihrer Position im Text
    top_sentences = sorted(top_sentences)

    # Gebe die extrahierten Schlüsselsätze zurück
    extracted_sentences = [sentences[index] for index in top_sentences]
    return extracted_sentences


def similarity_function(doc1, doc2):
    # Berechne die Cosinus-Ähnlichkeit zwischen den beiden Dokumenten
    similarity = doc1.similarity(doc2)
    return similarity




def compression_ratio(text, summary):
    # Berechne das Verhältnis der Anzahl der Wörter in der Zusammenfassung zur Anzahl der Wörter im Ausgangstext
    num_words_text = len(text.split())
    num_words_summary = len(summary.split())
    ratio = num_words_summary / num_words_text
    return ratio



def compression(text, compression_rate):
    max_iterations = 20
    iterations = 0
    #compression_rate -= 0.05
    
    extracted = textrank_extractive(text, compression_rate)
    summary = ' '.join(extracted)
    compression_rate_renwed = compression_rate


    while compression_ratio(text, summary) < compression_rate and iterations < max_iterations:
        iterations += 1
        compression_rate_renwed += 0.05
        if compression_rate_renwed > 1:
            compression_rate_renwed = 1
        extracted = textrank_extractive(text, compression_rate=compression_rate_renwed)
        summary = ' '.join(extracted)
    return summary




In [43]:
model_name='facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("text2text-generation", model=model_name)

In [44]:
def token_count(text):
    tokens = text.split()
    return len(tokens)


def adjust_length(text):
    length = token_count(text)
    if length < 80:
        min_length = length + int(length * 0.05)
        max_length = min_length + 50
    elif length < 100:
        min_length = length + int(length * 0.3)
        max_length = min_length + 100
    else:
        min_length = math.ceil(length / 50) * 70
        max_length = min_length + 100
    return min_length, max_length

In [45]:
def batch_sent(sentenc,splitt=180,split='. '):
    sentences = sentenc.split(split)
    # Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
    batches = []
    batch = []
    batch_len = 0
    for sentence in sentences:
        
        sentence_len = len(tokenizer.tokenize(sentence))
        if sentence_len + batch_len > splitt:
            if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 256 Tokens sind
                batches.append(batch)
                batch = [sentence]
                batch_len = sentence_len
            # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
        else:
            batch.append(sentence)
            batch_len += sentence_len
    batches.append(batch)
    return batches

In [93]:
def paraphrase_of_text(df_s,text_name='text',komp_name='reduction_multiplier',split='. '):

    df_summary_testing = pd.DataFrame(columns=['Zusammenfassung','Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate','länge Zusammenfassung','länge Ausgangstext'])
    # Teilen Sie den Text in Sätze --> für tatsächliche umsetzubng
    soplitting=True
    for _, row in tqdm(df_s.iterrows(), total=df_s.shape[0]):
     
        text = row[text_name]
        komp = row[komp_name]
        text_gesamt_list=[]

        for batch in tqdm(batch_sent(text,split=split), desc='Verarbeite Batches'):
            # Zusammenfügen der Sätze in einem Batch
            batch_text = '. '.join(batch)
            min_length_test, max_length_test = adjust_length(batch_text)
            ext_summary=summarizer(batch_text, max_length=int(round(max_length_test*komp,0)), min_length=int(round(min_length_test*komp,0)),length_penalty=100,num_beams=2)
            # Erstellen Sie einen DataFrame für die aktuellen Ergebnisse
           
            text_gesamt_list.append(ext_summary[0]['generated_text'])
        text_gesamt = '. '.join(text_gesamt_list)
        actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100
      
        df_current = pd.DataFrame({
            'Zusammenfassung': [text_gesamt],
            'Min_Kompressionsrate': [min_length_test],
            'Max_Kompressionsrate': [max_length_test],
            'Endgueltige_Kompressionsrate': [actual_compression_rate],
            'länge Zusammenfassung': [len(text_gesamt.split(' '))],
            'länge Ausgangstext': [len(text.split(' '))]
        
        })
       
        # Fügen Sie die Daten zum DataFrame hinzu
        df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)
      
    return df_summary_testing


In [27]:
def calculate_compression(df, total_tokens_col, current_tokens_col, desired_compression_rate):
    df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
    df['compression_difference'] = df[desired_compression_rate] - df['current_compression_rate']
    df['reduction_multiplier'] = df[desired_compression_rate] / df['current_compression_rate']
    return df

In [89]:
def text_rank_algo(df):
    df_return = pd.DataFrame(columns=['text','text_rank_text','tokens_gesamt', 'token_text_rank','desired_compression_rate','text_rank_compression_rate'])
    # Schleife zur Generierung der zufälligen Werte
    random.seed(10)
    for index, row in tqdm(df.iterrows()):
        text = row["text"]
        
        random_value = round(random.uniform(0.2, 0.8), 2)  # Zufälliger Wert zwischen 0.2 und 0.8 auf zwei Stellen nach dem Komma begrenzt
        text_rank_text= compression(text,random_value)
        compression_ratio_value=compression_ratio(text, compression(text,random_value))
        df_current = pd.DataFrame({
                'text':[text],
                'text_rank_text': [text_rank_text],
                'tokens_gesamt': [len(text.split(' '))],
                'token_text_rank': [len(text_rank_text.split(' '))],
                'desired_compression_rate': [random_value],
                'text_rank_compression_rate': [compression_ratio_value]
            })
            # Fügen Sie die Daten zum DataFrame hinzu
        df_return = pd.concat([df_return, df_current], ignore_index=True)
    return df_return


In [49]:
def execute_text_gen(df,split='. '):
    rank_df=text_rank_algo(df)
    df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
    df_sum=paraphrase_of_text(df_zwischen[['text_rank_text','reduction_multiplier']],text_name='text_rank_text',split=split)
    merged_df = pd.concat([df_sum, df_zwischen], axis=1)
    merged_df['ent_com_rate']=merged_df['länge Zusammenfassung'] / merged_df['tokens_gesamt']
    return merged_df

In [54]:
df_train.classification.unique()


array(['Scientific', 'news', 'reviews', 'story'], dtype=object)

In [90]:
df_training=df_train[df_train.classification=='news'].reset_index(drop=True)[['classification','text']][0:10]

In [94]:
df_sum=execute_text_gen(df_training,split='. ')

10it [01:00,  6.04s/it]
100%|██████████| 10/10 [00:37<00:00,  3.75s/it]


In [95]:
df_sum[['Zusammenfassung', 'Min_Kompressionsrate', 'Max_Kompressionsrate',
       'länge Zusammenfassung',
       'text','tokens_gesamt',
       'token_text_rank', 'desired_compression_rate',
       'text_rank_compression_rate', 'current_compression_rate',
      'reduction_multiplier', 'ent_com_rate']]

,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,länge Zusammenfassung,text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,reduction_multiplier,ent_com_rate
0,The Administration of Union Territory Daman an...,34,84,33,The Administration of Union Territory Daman an...,60,33,0.54,0.550000,0.55,0.981818,0.55
1,Malaika Arora slammed an Instagram user who tr...,40,90,39,Malaika Arora slammed an Instagram user who tr...,60,39,0.46,0.650000,0.65,0.707692,0.65
2,Bihar Health Minister defined virgin as being ...,36,86,34,The Indira Gandhi Institute of Medical Science...,60,35,0.55,0.583333,0.583333,0.942857,0.566667
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,42,92,22,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,60,40,0.32,0.666667,0.666667,0.48,0.366667
4,Hotels in Maharashtra will train their staff t...,53,103,38,Hotels in Maharashtra will train their staff t...,60,51,0.69,0.850000,0.85,0.811765,0.633333
5,"Man, 32, was found hanging inside the washroom...",50,100,42,A 32-year-old man on Wednesday was found hangi...,60,48,0.69,0.800000,0.8,0.8625,0.7
6,Delhi High Court reduced the compensation awar...,37,87,35,The Delhi High Court reduced the compensation ...,61,36,0.59,0.600000,0.590164,0.999722,0.57377
7,60-year-old Dalit woman was allegedly lynched ...,27,77,21,A 60-year old Dalit woman was allegedly lynche...,60,26,0.30,0.433333,0.433333,0.692308,0.35
8,The Pawan Hans helicopter was flying at a crit...,63,113,40,An inquiry by the Aircraft Accident Investigat...,61,60,0.51,1.000000,0.983607,0.5185,0.655738
9,Congress party has opened a bank called 'State...,43,93,30,The Congress party has opened a bank called 'S...,57,41,0.40,0.719298,0.719298,0.556098,0.526316
